In [ ]:
from defaults import *
from matplotlib.colors import to_rgba

In [ ]:
import sys
path_granular = '/orange/physics-dept/an.kandala/Biophysics_projeccts/granular_learner/granular_learner_ananth/granular_learner'
#append to PATH
sys.path.append(path_granular)
from granular_learner import Granular_learner
from helper_funcs import wrapper

In [ ]:
#INITIALIZATION
epsilon = 4
r1 = 1
r2 = 1.4*r1
Lx = 30
Ly = 15
inner_radii = [r1, r2]
prob = [0.4, 0.6]
#place input boundary spheres:
boundary_sphere_radius = 1
boundary_sphere_gap = 0 # gap between boundary spheres 
n_spheres = 100
learner = Granular_learner(Lx, Ly, inner_radii, boundary_sphere_radius, boundary_sphere_gap,epsilon)
learner.initialize_(n_spheres, prob)

In [ ]:
def force_func(rdiff_vec, cut_off, epsilon, alpha, sigma):
    rdiff = np.linalg.norm(rdiff_vec, axis = 1)
    force = (-epsilon/cut_off)*(1 - rdiff/cut_off)**(alpha-1)*(r_diff_vec/rdiff)
    return 

In [ ]:
inner_positions = learner.inner_spheres[:, :-1]
cut_offs = learner.inner_cutoffs
inner_positions[:, 1] = wrapper(inner_positions[:, 1], Ly) #periodic boundary conditions in the y direction
#calculate the bulk energy:
uti = np.triu_indices(len(inner_positions), k=1) #obtain unique indices of the inner sphere combinations
r_diff_vec = inner_positions[uti[0]] - inner_positions[uti[1]] #calculate the difference in positions
r_diff_vec[:, 1] = r_diff_vec[:, 1] + Ly*(r_diff_vec[:, 1] < -Ly/2) - Ly*(r_diff_vec[:, 1] > Ly/2) #shortest distance in the y direction
epsilon = 2; alpha = 2
print(r_diff_vec)
r_diff = np.linalg.norm(r_diff_vec, axis = 1)
print(r_diff)
neighbors = np.where(r_diff < cut_offs)[0]
r_diff = r_diff[neighbors]; r_diff_vec = r_diff_vec[neighbors]; cut_offs = cut_offs[neighbors] #selecting only the neighbors
force_magnitudes = (epsilon/cut_offs)*(1 - r_diff/cut_offs)**(alpha-1)
force_directions = -r_diff_vec/r_diff[:, None]
forces = force_magnitudes[:, None]*force_directions
f = np.column_stack((neighbors, forces))
print(f)

In [ ]:
print(learner.boundary_cutoffs.shape)

In [61]:
#Calculate forces between the inner spheres and the boundary spheres:
boundary_positions = learner.boundary_spheres[:, :-1]
boundary_cutoffs = learner.boundary_cutoffs
r_diff_vec = inner_positions[:, np.newaxis, :] - boundary_positions[np.newaxis, :, :] #calculate the difference in positions between inner and boundary spheres
#r_diff_vec.shape = (inner_spheres, boundary_spheres, 2)
r_diff_vec[:, :, 1] = r_diff_vec[:, :, 1] + Ly*(r_diff_vec[:, :, 1] < -Ly/2) - Ly*(r_diff_vec[:, :, 1] > Ly/2) #calculate the shortest distance in the y direction
r_diff = np.linalg.norm(r_diff_vec, axis = 2) #calculate the distance between the inner and boundary spheres
print(r_diff.shape)
neighbors = np.where(r_diff < boundary_cutoffs) #selecting the neighbors
r_diff = r_diff[neighbors]; r_diff_vec = r_diff_vec[neighbors]; boundary_cutoffs = boundary_cutoffs[neighbors] #selecting only the neighbors
#r_diff, boundary_cutoffs are now 1d arrays with len=num of neighbors| r_diff_vec is a 2d array 
force_magnitudes = (epsilon/boundary_cutoffs)*(1 - r_diff/ boundary_cutoffs)**(alpha-1)
force_directions = -r_diff_vec/r_diff[:, None]
forces = force_magnitudes[:, None]*force_directions
f = np.column_stack((neighbors[0], neighbors[1], forces))
print(f)


(200, 16)
[[ 6.00000000e+00  1.10000000e+01  9.57912447e-02 -1.36844635e-02]
 [ 1.00000000e+01  1.30000000e+01  2.86402066e-02 -6.49178016e-02]
 [ 1.00000000e+01  1.40000000e+01  5.53476691e-01  2.21390676e-01]
 [ 1.30000000e+01  1.20000000e+01  9.28282699e-02  1.54713783e-02]
 [ 1.40000000e+01  1.20000000e+01  2.25000000e-01 -3.00000000e-01]
 [ 1.40000000e+01  1.30000000e+01  2.25000000e-01  3.00000000e-01]
 [ 1.50000000e+01  5.00000000e+00 -4.82004672e-01 -1.87446261e-01]
 [ 1.50000000e+01  6.00000000e+00 -2.88521307e-02  5.28955729e-02]
 [ 1.80000000e+01  1.00000000e+01  2.55953574e-01 -3.24207860e-01]
 [ 1.80000000e+01  1.10000000e+01  2.23948495e-01  3.13527893e-01]
 [ 1.90000000e+01  4.00000000e+00 -2.44970610e-01 -2.72189566e-01]
 [ 1.90000000e+01  5.00000000e+00 -2.44970610e-01  2.72189566e-01]
 [ 3.10000000e+01  9.00000000e+00  9.83235618e-02 -1.17051859e-02]
 [ 4.50000000e+01  2.00000000e+00 -3.92187565e-02  6.97222338e-03]
 [ 5.00000000e+01  0.00000000e+00 -3.67151479e-02  7

In [ ]:
array1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
array2 = np.array([[0, 2, 3], [4, 1, 6], [7, 8, 2]])
cond = np.where(array1 > array2)
print(cond)

In [ ]:
#select two points at random between ([0, Lx], [0, Ly])
num_points = 4
points = np.column_stack((np.random.uniform(0, Lx, num_points), np.random.uniform(0, Ly, num_points)))
uti = np.triu_indices(num_points, k=1)
r_diff = points[uti[1]] - points[uti[0]]
r_diff[:, 1] = r_diff[:, 1] + Ly*(r_diff[:, 1] < -Ly/2) - Ly*(r_diff[:, 1] > Ly/2)
print(Ly*(r_diff[:, 1] < -Ly/2))
# print(r_diff)
r_diff_norm = np.linalg.norm(r_diff, axis = 1)
r_diff_unit_vecs = r_diff/r_diff_norm[:, None]
# print(r_diff_unit_vecs)


# #plot rdiff
# ax.plot([points[0, 0], points[0, 0] + r_diff[0]], [points[0, 1], points[0, 1] + r_diff[1]], '-o', lw=1, color='red')


In [ ]:
fig, ax = plt.subplots(dpi=300)
ax.set_xticks([]) #remove axes
ax.set_yticks([])
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
#generate num_points colors:
colors = {i: to_rgba(f"C{i}") for i in range(num_points)}
for i in range(num_points):
    ax.plot(points[i, 0], points[i, 1], 'o', color=colors[i])
ax.plot(points[:, 0], points[:, 1], '-', color='black', lw=0.1)
for ll, r in enumerate(r_diff_unit_vecs):
    [x_start, y_start] = points[uti[0][ll]] 
    [dx, dy] = -r
    color = colors[uti[1][ll]]
    ax.arrow(x_start, y_start, dx, dy, head_width=0.05, head_length=0.1, fc=color, ec=color)
    [x_start, y_start] = points[uti[1][ll]] 
    [dx, dy] = r
    color = colors[uti[0][ll]]
    ax.arrow(x_start, y_start, dx, dy, head_width=0.05, head_length=0.1, fc=color, ec=color)
fig.savefig('test_forces.png')

In [ ]:
unit_vectors = r_diff/np.linalg.norm(r_diff, axis = 1)[:, None]

In [ ]:
# Example 2D array
array_2d = np.array([[1, 2, 3],
                     [4, 5, 6],
                     [7, 8, 9]])

# Example 1D array
array_1d = np.array([2, 3, 4])

# Divide each row of the 2D array by the corresponding entry of the 1D array
result_array = array_2d / array_1d[:, None]

print("Original 2D array:")
print(array_2d)

print("\nOriginal 1D array:")
print(array_1d)

print("\nResulting array:")
print(result_array)